In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
!pip install Wikipedia-API


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!!pip install -U transformers

['Requirement already satisfied: transformers in c:\\python312\\lib\\site-packages (4.57.3)',
 'Requirement already satisfied: filelock in c:\\python312\\lib\\site-packages (from transformers) (3.18.0)',
 'Requirement already satisfied: huggingface-hub<1.0,>=0.34.0 in c:\\python312\\lib\\site-packages (from transformers) (0.34.3)',
 'Requirement already satisfied: numpy>=1.17 in c:\\python312\\lib\\site-packages (from transformers) (2.1.3)',
 'Requirement already satisfied: packaging>=20.0 in c:\\python312\\lib\\site-packages (from transformers) (24.2)',
 'Requirement already satisfied: pyyaml>=5.1 in c:\\python312\\lib\\site-packages (from transformers) (6.0.2)',
 'Requirement already satisfied: regex!=2019.12.17 in c:\\python312\\lib\\site-packages (from transformers) (2025.7.34)',
 'Requirement already satisfied: requests in c:\\python312\\lib\\site-packages (from transformers) (2.32.4)',
 'Requirement already satisfied: tokenizers<=0.23.0,>=0.22.0 in c:\\python312\\lib\\site-packag

In [ ]:
import wikipediaapi

wiki_wiki = wikipediaapi.Wikipedia('FreeBirds Crew (freebirdscrew@gmail.com)','en')

def get_wikipedia_page(title):
    page_py = wiki_wiki.page(title)
    if not page_py.exists():
        return None
    return page_py.text

page_title = "Python (programming language)"
wikipedia_data = get_wikipedia_page(page_title)

with open("wikipedia_data.txt", "w", encoding="utf-8") as file:
    file.write(wikipedia_data)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

c:\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dell\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [8]:
# Prepare data for fine-tuning
train_data = TextDataset(
    tokenizer=tokenizer,
    file_path="wikipedia_data.txt",
    block_size=128
)

In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)
data_collator

DataCollatorForLanguageModeling(tokenizer=GPT2Tokenizer(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
), mlm=False, whole_word_mask=False, mlm_probability=0.15, mask_replace_prob=0.8, random_replace_prob=0.1, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt', seed=None)

In [ ]:
training_args = TrainingArguments(
    output_dir="./fine-tuned_model",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data
)

In [11]:
trainer.train()

c:\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=51, training_loss=3.272614422966452, metrics={'train_runtime': 832.8659, 'train_samples_per_second': 0.234, 'train_steps_per_second': 0.061, 'total_flos': 12737986560000.0, 'train_loss': 3.272614422966452, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./fine-tuned_model_gpt2_wiki")

tokenizer.save_pretrained("./fine-tuned_model_gpt2_wiki_token")

('./fine-tuned_model_gpt2_wiki_token\\tokenizer_config.json',
 './fine-tuned_model_gpt2_wiki_token\\special_tokens_map.json',
 './fine-tuned_model_gpt2_wiki_token\\vocab.json',
 './fine-tuned_model_gpt2_wiki_token\\merges.txt',
 './fine-tuned_model_gpt2_wiki_token\\added_tokens.json')

In [ ]:
from transformers import pipeline

fine_tuned_model = GPT2LMHeadModel.from_pretrained("./fine-tuned_model_gpt2_wiki")
fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./fine-tuned_model_gpt2_wiki_token")

generator = pipeline('text-generation', model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)
generated_text = generator("What is GenAI?", max_length=100, num_return_sequences=1)[0]['generated_text']

print(generated_text)

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


What is GenAI?

GenAI was developed by the World Wide Web Consortium (WWW), along with the Internet Archive (IA), which maintains and compiles the HTTP/2 protocol. GenAI is a Python-based programming language, which is part of the Python community. It supports a range of development platforms including Python, Ruby, and C.

GenAI was developed as a way to support research in the field of artificial intelligence. It is currently the most widely used programming language, with more than 3 billion users.

Why is GenAI relevant to programming?

GenAI is a reflection of the programming philosophy of the Python community (including the current Python 3.8 and Python 3.7), which emphasizes greater abstraction, modularity, and modularity over traditional programming languages.

GenAI is a general purpose language that can be used for many different tasks, including machine learning (ML), machine learning (ML-R), and machine learning and machine learning applications.

How can I use GenAI to lea